In [3]:
# Imports and add-ons

import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import time
import pickle

# Importing pathlib package to allow parsing of csv files
from pathlib import Path
import sys

In [5]:
# Opening the raw data file as a PATH object
raw_data_path = Path(r"farelli_phosphatases_raw_data.xlsx").parent.absolute() / Path("farelli_phosphatases_raw_data.xlsx")

# Reading excel file from PATH object
raw_data_file = pd.read_excel(raw_data_path)

# Making variable which contains list of sheets available
xl = pd.ExcelFile(raw_data_path)
Sheet_Names = xl.sheet_names
# print(Sheet_Names)

# Make list of EFI IDs based on what is in excel file sheets
EFI_ID_List = [int(x) for x in Sheet_Names if x.isdigit()]
# print(EFI_ID_List)

# Make list of EFI IDs based on sheet named Screen List for comparison
Screen_List = pd.read_excel(raw_data_path, sheet_name='Screen List', header=None)
Screen_List = Screen_List.to_numpy()
Screen_List = Screen_List.transpose()
Screen_List = Screen_List.tolist()
Screen_List = Screen_List[0]
# print(Screen_List)

# Compares to ensure that all sheets in excel file matches what is in Screen List
print('\n' + 'Is all the data claimed to be recorded present: ' + str(Screen_List == EFI_ID_List))

# prints length of both vectors
print('\n' + 'Length of Screen List Vector: ' + str(len(Screen_List)))
print('\n' + 'Length of EFI ID List Vector: ' + str(len(EFI_ID_List)))


Is all the data claimed to be recorded present: True

Length of Screen List Vector: 218

Length of EFI ID List Vector: 218


In [ ]:
GI

In [ ]:
def get_protein_sequence(GI_Num):
    '''
    This function obtains the protein sequence using a given GI # or GI ID
    and returns the protein sequence as a string
    '''

    # First Must Construct a URL
    '''
    3 Main parts of a URL required to use E-utilities and access NCBI database
        1. Base URL: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/
        2. Utility Name: (eg: ESearch) More Here-> https://dataguide.nlm.nih.gov/eutilities/utilities.html
        3. Parameters: Details of query such as name of database,l search terms, format of results, etc

    Combining the 3 parts (example) gives us:
        1. Base URL: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/
        2. Utility Name: ESearch = esearch.fcgi?
        3. Parameters: db=pubmed&term=nature[journal]+AND+3D+printing

    list of db allowed = https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    Result: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=nature[journal]+AND+3D+printing
    '''

    # We can use GI # as the UID so lets fetch the data from it
    Base_URL = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    Utility_Name = 'efetch.fcgi?'
    Parameters = 'db=protein&id=' + str(GI_Num) + '&retmode=xml'
    URL_request = Base_URL + Utility_Name + Parameters
    URL_request = URL_request.replace(' ','+') # Need to replace all spaces within URL for query
    
    response = requests.get(URL_request, auth=('anaqi.afendi@mail.utoronto.ca', '83ccb7c8bafa4df9a8120b43ecb179e92809')) # Stores response of URL request

    '''
    # Convert bytes of response.content ot string to import to xml
    parser = ET.XMLParser(encoding='utf-8')
    root = ET.fromstring(response.content, parser=parser)
    protein_sequence = root.find('GBSeq').find('GBSeq_sequence').text
    return(protein_sequence)
    '''

    # Convert bytes of response.content to string to import to xml
    bytes_string = str(response.content, 'utf-8')
    root = ET.fromstring(bytes_string)
    protein_sequence = root.find('GBSeq').find('GBSeq_sequence').text 
    return(protein_sequence)

    '''
    # Try different method
    tree = ET.parse(response.content)
    root = tree.getroot
    protein_sequence = root.find('GBSeq').find('GBSeq_sequence').text
    return(protein_sequence)
    '''

In [7]:
'''
Dataframe layout: 218 columns (proteins), 168 rows (metabolites)
This section constructs the data frame
'''

# Protein GI # = Column header, obtain Protein Sequence and map using dictionary

# Metabolites stored in 14 x 12 array as comments on top of activity data
# Need to make a vector of length 168 containing metabolite name

from openpyxl import Workbook
from openpyxl import load_workbook

wb = wb = load_workbook(raw_data_path)
ws = wb["501030"] # or whatever sheet name

metabolite_list = [] # Store metabolite names
metabolite_dict = {} # Store metabolites in dictionary to map to original excel grid layoutz

for r in range(7,21):
    for c in range(3,15):
        whole_comment = ws.cell(row=r,column=c).comment
        metabolite = whole_comment.text.strip("\n")
        metabolite_list.append(metabolite)
        
        letter = ws.cell(row=r,column=c).column_letter
        number = ws.cell(row=r,column=c).row
        metabolite_dict[str(letter)+str(number)] = metabolite

# Establish initial variables
activity_df = pd.DataFrame(columns=EFI_ID_List)
GI_dict = {} # maps GI to EFI_ID, EFI_ID is key
Protein_name_dict = {} # maps protein name to EFI_ID, EFI_ID is key

# For loop to store all activitiy data into one dataframe
for EFI_ID in EFI_ID_List:
    data_frame = pd.read_excel(raw_data_path, sheet_name=str(EFI_ID), header=0)
    
    GI_Num = data_frame['GI #'][0]
    if type(GI_Num) != str:
        GI_Num = str(int(GI_Num))
    GI_dict[EFI_ID] = GI_Num # Add GI Num to dictionary
    
    Protein_Name = str(data_frame['Name and Species'][0])
    Protein_name_dict[EFI_ID] = Protein_Name # Add Protein name to dictionary

    activities = []
    for r in range(5,19):
        for c in range(2,14):
            activity = data_frame.iloc[r,c]
            activities.append(activity)
    
    activity_df[EFI_ID] = activities

# Return Name_and_species for searching through NCBI Database
# Protein_Name = str(data_frame['Name and Species'][0])
# print(Protein_Name)


In [ ]:
EFI_ID = EFI_ID_List[0]
data_frame = pd.read_excel(raw_data_path, sheet_name=str(EFI_ID), header=0)
activities = []
for r in range(5,19):
    for c in range(2,14):
        activity = data_frame.iloc[r,c]
        activities.append(activity)
        print(str(r) + "," + str(c))


In [ ]:
# GI_Num = GI_dict[502334]
GI_Num = 'Q836C7'
print(get_protein_sequence(GI_Num=GI_Num))

In [ ]:
'''This section adds the protein_sequences to a dictionary from NCBI, it doesn't work atm'''
# protein_dict = { get_protein_sequence(i) for i in EFI_ID_List }

Protein_seq_dict = {} # maps protein sequence to EFI_ID, EFI_ID is key
GI_Error_dict = {} # maps EFI_ID as key to GI # which returned errors

def divide_chunks(long_list, num):
    # looping till length l 
    for i in range(0, len(long_list), num):  
        yield long_list[i:i + num] 

chunk_list = list(divide_chunks(long_list=EFI_ID_List,num=30))

for sub_chunk in range(0,len(chunk_list)):
    print("subchunk: " + str(sub_chunk))
    for EFI_ID in chunk_list[sub_chunk]:
        GI_Num = GI_dict[EFI_ID]
        try:
            # print(get_protein_sequence(GI_Num))
            protein_sequence = get_protein_sequence(GI_Num)
            Protein_seq_dict[EFI_ID] = protein_sequence
            print("Success with EFI_ID: " + str(EFI_ID) + ", GI_Num: " + str(GI_Num))
        except Exception:
            GI_Error_dict[EFI_ID] = GI_Num
            print("Error with EFI_ID: " + str(EFI_ID) + ", GI_Num: " + str(GI_Num))
        time.sleep(2)
    print("\n")


In [ ]:
# Manually add proteins
Protein_seq_dict[508512] = "MYQVVASDLDGTLLSPDHFLTPYAKETLKLLTARGINFVFATGRHYIDVGQIRDNLGIRSYMITSNGARVHDSDGQQIFAHNLDRDIAADLFEIVRNDPKIVTNVYREDEWYMNRHRPEEMRFFKEAVFNYKLYEPGELDPQGISKVFFTCEDHEHLLPLEQAMNARWGDRVNVSFSTLTCLEVMAGGVSKGHALEAVAKMLGYTLSDCIAFGDGMNDAEMLSMAGKGCIMANAHQRLKDLHPELEVIGSNADDAVPRYLRKLYLD"

In [ ]:
# Save important information into pickle files and csv under "Processed Data" folder
dirName = Path(r"farelli_phosphatases_raw_data.xlsx").parent.absolute() / Path("Processed Data")
try:
    os.makedirs(dirName)
    print("Directory " , dirName, " Created ")
except FileExistsError:
    print("Directory " , dirName, " Already Exists ")

pickle.dump(EFI_ID_List, open(dirName / Path('EFI_ID_List.p'),'wb'))
pickle.dump(Protein_seq_dict, open(dirName / Path('Protein_seq_dict.p'),'wb'))
pickle.dump(metabolite_dict, open(dirName / Path('metabolite_dict.p'),'wb'))
activity_df.to_csv(dirName / Path('activations.csv'))

In [ ]:
for EFI_ID in GI_Error_dict.keys():
    data_frame = pd.read_excel(raw_data_path, sheet_name=str(EFI_ID), header=0)
    Name = data_frame['Name and Species'][0]
    print(Name)

In [8]:
dirName = Path(r"farelli_phosphatases_raw_data.xlsx").parent.absolute() / Path("Processed Data")
pickle.dump(GI_dict, open(dirName / Path('GI_dict.p'),'wb'))

In [9]:
# Load the clean data into variables
DataPath = "Processed Data/"


with open(DataPath + 'EFI_ID_List.p', 'rb') as EFI_ID:
    EFI_ID_List = pickle.load(EFI_ID)

In [14]:
# Make a section to convert FASTA aligned sequences into dictionary
with open('Processed Data/'+'Less Than 300.aln.txt') as f:
    textlines = list(f)

Protein_aligned_dict = {}
for ID in EFI_ID_List:
        
    Aligned_Sequence = ''
        
    # Loop over each line in text and check if the beginning matches the EFI_ID
    for line in textlines:
        if line.startswith(str(ID)):
            line = line[0:67] # Only first 67 characters contain sequence info
            line = line.strip(str(ID))
            line = line.strip(' ')

            Aligned_Sequence = Aligned_Sequence + line
        
    Protein_aligned_dict[ID] = Aligned_Sequence # Update dictionary


# Dump dictionary into this path
dirName = Path(r"farelli_phosphatases_raw_data.xlsx").parent.absolute() / Path("Processed Data")


In [30]:
len(Protein_aligned_dict[EFI_ID_List[0]])

769

In [29]:
len('------------------------------------------------------------')

60